# Aim
Optimize 55 20 node graphs, 5 for each even degree node count, with same degree sequence but different that the old 55

Imports

In [1]:
import warnings
warnings.filterwarnings('ignore')

#################
# My notebook issues
import os
import sys
nb_dir = os.path.split(os.getcwd())[0]
if nb_dir not in sys.path:
    sys.path.append(nb_dir)
sys.path.append('home/egupta/.local/bin')

##############

import os
import numpy as np
import scipy.stats as ss
import networkx as nx
import csv 
import numpy as np
import seaborn as sns
from scipy.optimize import curve_fit
import matplotlib
from matplotlib import pyplot as plt

%matplotlib inline


##qtensor imports
import qtensor
from qtensor import QAOA_energy
from qtensor import parameter_optimization as popt
import torch

2021-08-02 08:17:32,400- WARNING•	QuickBB solver is unavailable
2021-08-02 08:17:32,402- WARNING•	Tamaki solver is unavailable: No path /home/egupta/.local/lib/python3.7/site-packages/thirdparty/tamaki_treewidth


#### Important Functions

In [2]:
def get_qaoa_params(graph, edge=None, gamma=[0.5], beta=[0.5], steps=50):
    loss_history, params_history = popt.qaoa_maxcut_torch(graph, gamma, beta,
                                                          ordering_algo='greedy',
                                                          Opt=torch.optim.RMSprop,
                                                          edge=edge,
                                                          opt_kwargs=dict(lr=0.002),
                                                          steps=steps,
                                                          pbar=False)
    best_ix = np.argmin(loss_history)
    return params_history[best_ix], loss_history[best_ix]

def get_edge_contribution(graph, edge, gamma, beta):
    sim = qtensor.QtreeSimulator()
    composer = qtensor.DefaultQAOAComposer(graph, gamma=gamma, beta=beta)
    composer.energy_expectation_lightcone(edge)
    return np.real(sim.simulate_batch(composer.circuit)[0])

def opt_runs(G, seeds = 20, steps = 200):
  """
  Doing multiple optimization runs, each with random initialized param
  Returning results which inlclude the optimal param and corresponding energies
  """

  #first do 20 optimization runs 
  s = 0
  g_results = []

  while s<seeds:
    print('seed #' + str(s))
    #initialize param
    gamma = [np.random.rand()]
    beta = [np.random.rand() ]
    initial_energy = QAOA_energy(G,gamma =  gamma, beta = beta)

    #optimization
    params, loss = get_qaoa_params(G, gamma=gamma, beta=beta,edge = None, steps=steps)
    #print('Finished get_qaoa_params')
    gamma_opt, beta_opt = params[0], params[1]
    opt_energy = QAOA_energy(G, gamma = gamma_opt, beta = beta_opt)

    result = np.asarray([float(gamma_opt), float(beta_opt), float(opt_energy), float(initial_energy)])
    g_results.append(result)
    s+=1

  
  return g_results

#### Import New Graphs

In [10]:
file3 = open("New20NodeRandomGraphs.txt")
mat_list = np.loadtxt(file3).reshape(55,20,20)

def mats_to_graphs(mat_list):
  """
  Given list of adjaceny matrix repr of graphs, converts to actual graphs
  """
  g_list = []
  for mat in mat_list:
    arr = np.array(mat)
    G = nx.convert_matrix.from_numpy_matrix(arr)
    g_list.append(G)
  return g_list

g_list = mats_to_graphs(mat_list)

In [13]:
n_g_list = g_list[0:27]

# Optimize Graphs

In [16]:
results = []
for g_idx in range(len(n_g_list)):
    print(g_idx)
    result = opt_runs(n_g_list[g_idx], seeds = 20, steps= 200)
    print(result)
    results.append(result)

0
seed #0



seed #1



seed #2



seed #3



seed #4


...


seed #16



seed #17



seed #18



seed #19



[array([ 0.91387689,  0.61604851, 20.65191211, 16.81542054]), array([ 0.91388339,  0.61627734, 20.65193418, 14.76457029]), array([ 0.44355792,  0.11344278, 17.014426  , 15.96679592]), array([ 0.0861156 ,  0.8837207 , 20.6519342 , 15.25643248]), array([ 0.08610661,  0.38372058, 20.65193424, 16.83258451]), array([ 0.55642414,  0.88650948, 17.01442623, 16.93165574]), array([ 0.08610662,  0.38372055, 20.65193424, 14.16972218]), array([ 0.9138934 ,  0.61628133, 20.65193424, 13.43024558]), array([ 0.91388935,  0.11627942, 20.65193423, 16.84724501]), array([ 0.08610562,  0.38372064, 20.65193424, 15.30279843]), array([ 0.91443419,  0.61627913, 20.65178219, 16.01056672]), array([ 0.91386813,  1.11627948, 20.65193391, 14.71829881]), array([ 0.91389096,  0.61627942, 20.65193424, 16.6102487 ]), array([ 0.44356108,  0.11348453, 17.01442628, 16.89712837]), array([ 0.08610662,  0.38372058, 20.65193424, 14.98917535]), array([ 0.91393858,  1.11627841, 20.65193318, 14.58023924]), array([ 0.08614699,  0

In [17]:
a_file = open("New20NodeRandomGraphsData1.txt", "w")
for result in results:
    for row in result:
        np.savetxt(a_file, row)
a_file.close()